# Lab 6: Multiple Linear Regression and Feature Engineering

** This lab is due Monday October 1, 2018 at 11:59pm (graded on accuracy) **

In this lab we will work through the process of:
1. implementing a linear model, defining loss functions, 
1. feature engineering,
1. minimizing loss functions using numeric libraries 

This lab will continue using the toy tip calculation dataset.

# Collaborators

Write names in this cell:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
np.random.seed(42)
plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")
%matplotlib inline

# Loading the Tips Dataset

To begin with, we load the tips dataset from the `seaborn` library.  The tips data contains records of tips, total bill, and information about the person who paid the bill.

In [2]:
data = sns.load_dataset("tips")

print("Number of Records:", len(data))
data.head()

Number of Records: 244


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# Question 1: Defining the Model and Feature Engineering

In the previous lab, we defined a simple linear model with only one parameter. Now let's make a more complicated model that utilizes other features in our dataset. Let our prediction for tip be a combination of the following features:

$$
\text{Tip} = \theta_1 * \text{total_bill} + \theta_2 * \text{sex} + \theta_3 * \text{smoker} + \theta_4 * \text{day} + \theta_5 * \text{time} + \theta_6 * \text{size}
$$

Notice that some of these features are not numbers! But our linear model will need to predict a numerical value. Let's start by converting some of these non-numerical values into numerical values. Below we split the tips and the features.

In [3]:
tips = data['tip']
X = data[data.columns.difference(['tip'])]

## Question 1a: Feature Engineering
First, let's convert everything to numerical values. A straightforward approach is to map some of these non-numerical features into numerical ones. For example, we can treat the day as a value from 1-7. However, one of the disadvantages in directly translating to a numeric value is that we unintentially assign certain features disproportionate weight. Consider assigning Sunday to the numeric value of 7. Monday is assigned to 1 and thus Sunday has 7 times the influence of Monday in our linear model which can lower the accuracy of our model.

Instead, let's do one-hot encoding! 

As discussed in lecture, one-hot encoding will produce a binary vector indicating the non-numeric feature. Sunday would be encoded as a [0 0 0 0 0 0 1]. This assigns a more even weight across each category in non-numeric features. Complete the code below to one-hot encode our dataset.

In [4]:
def one_hot_encode(data):
    """
    Return the one-hot encoded dataframe of our input, data. 
    Hint: check pd.get_dummies
    """
    ### BEGIN SOLUTION
    return pd.get_dummies(data)
    ### END SOLUTION

In [5]:
one_hot_X = one_hot_encode(X)
one_hot_X.head()

,size,total_bill,day_Thur,day_Fri,day_Sat,day_Sun,sex_Male,sex_Female,smoker_Yes,smoker_No,time_Lunch,time_Dinner
0,2,16.99,0,0,0,1,0,1,0,1,0,1
1,3,10.34,0,0,0,1,1,0,0,1,0,1
2,3,21.01,0,0,0,1,1,0,0,1,0,1
3,2,23.68,0,0,0,1,1,0,0,1,0,1
4,4,24.59,0,0,0,1,0,1,0,1,0,1


## Question 1b: Defining the Model
Now that all of our data is numeric, let's define our model function.

In [6]:
def linear_model(thetas, X):
    """
    Return the linear combination of thetas and features as defined above.
    """
    ### BEGIN SOLUTION
    return X.dot(thetas)
    ### END SOLUTION

In [7]:
assert linear_model(np.arange(1,5), np.arange(1,5)) == 30

# Question 2: Fitting the Model: Numeric Methods
Recall in the previous lab we defined multiple loss functions and found optimal theta using the scipy.minimize function. Adapt the loss functions and optimization code from the previous lab (provided below) to work with our new linear model.

In [8]:
from scipy.optimize import minimize

def l1(y, y_hat):
    return np.abs(y-y_hat)

def l2(y, y_hat):
    return (y - y_hat)**2

def minimize_average_loss(loss_function, model, x, y):
    """
    loss_function: either the squared or absolute loss functions from above.
    model: the model (as defined above)
    x: the x values (one-hot encoded data)
    y: the y values (tip amounts)
    return the estimate for each theta as a vector
    
    Note we will ignore failed convergence for this lab ... 
    """
    
    ## Notes on the following function call which you need to finish:
    # 
    # 0. the ... should be replaced with the average loss evaluated on 
    #       the data x, y using the model and appropriate loss function
    # 1. x0 are the initial values for THETA.  Yes, this is confusing
    #       but optimization people like x to be the thing they are 
    #       optimizing.
    # 2. We extract the 'x' entry in the dictionary which corresponds
    #       to the value of thetas at the optimum
    ### BEGIN SOLUTION
    return minimize(lambda theta: loss_function(model(theta, x), y).mean(), x0=np.zeros(12))['x']
    ### END SOLUTION
    return minimize(lambda theta: ..., x0=...)

minimize_average_loss(l2, linear_model, one_hot_X, tips)

array([ 0.17599118,  0.09448701,  0.01519965,  0.17746084,  0.05600945,
        0.1519934 ,  0.18411098,  0.21655245,  0.15712745,  0.24353541,
        0.23440142,  0.16626169])

In [9]:
expected_l1 = np.array([0.08606582,0.10209183,-0.05485151,0.20643915,-0.03693622,0.28502352,0.117988,0.28168694,0.13459131,0.26508361,0.17577437,0.22390056])
expected_l2 = np.array([[0.17599118,0.09448701,0.01519965,0.17746084,0.05600945,0.1519934,0.18411098,0.21655245,0.15712745,0.24353541,0.23440142,0.16626169]])
assert np.isclose(np.array(minimize_average_loss(l1, linear_model, one_hot_X, tips)), expected_l1).all()
assert np.isclose(np.array(minimize_average_loss(l2, linear_model, one_hot_X, tips)), expected_l2).all()

# Question 3: Fitting the Model: Analytic Methods
Let's also fit our model analytically, for the l2 loss function. In this question we will derive an analytical solution, fit our model and compare our results with our numerical optimization results.

## Question 3a: Least Squares Solution
Recall that if we're fitting a linear model with the l2 loss function, we are performing least squares! Remember, we are solving the following optimization problem for least squares:

$$\min_{\theta} ||X\theta - y||^2$$

Let's begin by deriving the analytic solution to least squares. Write your answer in LaTeX in the cell below. Assume X is full column rank.

### BEGIN SOLUTION
Take the gradient and set theta equal to 0. 

$$2(X\theta - y) = 0$$
$$ X\theta = y$$
$$ X^T X\theta = X^T y$$
$$ \theta = (X^TX)^{-1} X^Ty$$
### END SOLUTION

## Question 3b: Solving for Theta
Now that we have the analytic solution for $\theta$, let's find the optimal numerical thetas for our tips dataset. Fill out the function below. Make sure you use the float type in your calculations using .astype(float) and use the np.linalg.inv function.

In [10]:
def get_analytical(x, y):
    """
    x: our one-hot encoded dataset
    y: tip amounts
    """
    ### BEGIN SOLUTION
    xTx = x.T.dot(x)
    xTy = x.T.dot(y)
    #return np.linalg.solve(xTx, xTy) this is better than inverse
    return np.linalg.inv(xTx).dot(xTy)
    ### END SOLUTION

In [11]:
analytical_thetas = get_analytical(one_hot_X.astype(float), tips.astype(float))
print("Our analytical loss is: ", l2(linear_model(analytical_thetas, one_hot_X),tips).mean())
print("Our numerical loss is: ", l2(linear_model(minimize_average_loss(l2, linear_model, one_hot_X, tips), one_hot_X), tips).mean())

Our analytical loss is:  59.7334147541
Our numerical loss is:  1.01035356124


In [12]:
assert np.isclose(l2(linear_model(analytical_thetas, one_hot_X),tips).mean(), 59.733414754098362)

## Question 3c: Weird Results?
Our analytical loss is surprisingly much worse than our numerical loss. Why is this? 

Hint: https://stackoverflow.com/questions/31256252/why-does-numpy-linalg-solve-offer-more-precise-matrix-inversions-than-numpy-li

### BEGIN SOLUTION
np.linalg.inv loses precision, which propogates error throughout the calculation. If you're not convinced, try not using .astype(float) and seeing the error. It's much more, around 1800. Similarly, try np.linalg.solve instead of np.linalg.inv, you'll find that our loss is much closer to expected. These results is meant to demonstrate that even if our math is correct, limits of our computational precision and machinery can lead us to poor results.
### END SOLUTION